In [20]:

import numpy as np
from sklearn.linear_model import ElasticNet
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

##### Data loading

In [21]:
# Path to the data files
data_path_1 = '../data/case1Data.csv'

# Load the data into a numpy array
data_np = np.loadtxt(data_path_1, delimiter=',', skiprows=1)

# Create a pandas dataframe and use the first row as the column names
data_pd = pd.read_csv(data_path_1, sep=',', header=0)


# Splitting the data into features and target
X = data_pd.iloc[:, 1:]
y = data_pd.iloc[:, 0]

print("X: ", X.shape)
print("y: ", y.shape)


# Define which columns are continuous and categorical
# Example: assuming all features are continuous
# If you have categorical columns, update accordingly
categorical_cols = list(range(X.shape[1]-5, X.shape[1]))  # Last 5 columns as categorical
continuous_cols = list(range(X.shape[1] - 5))  # All columns before the last 5 as continuous

X:  (100, 100)
y:  (100,)


In [22]:
# Define preprocessing steps for continuous and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', KNNImputer(n_neighbors=5)),  # KNN Imputer for continuous features
            ('scaler', StandardScaler())  # Standardization of continuous features
        ]), continuous_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute categorical missing values
            ('encoder', OneHotEncoder(handle_unknown='ignore'))  # One hot encode categorical features
        ]), categorical_cols)
    ]
)

In [23]:

# Define the classifier (example with RandomForest)
regressor = ElasticNet()

# Create a pipeline that first applies preprocessing and then fits the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

In [27]:
# Nested Cross-Validation (Outer loop for model evaluation, Inner loop for hyperparameter tuning)
# Inner loop for hyperparameter tuning
param_grid = {
    'regressor__alpha': [0.1, 1, 10, 100],  # Regularization strength
    'regressor__l1_ratio': [.1, .5, .7, .9, .95, .99, 1]  # Mix between Lasso and Ridge
}

# Outer loop for cross-validation (use KFold for regression)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Inner loop for GridSearchCV (hyperparameter tuning)
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)

# Evaluate with cross_val_score (outer loop)
nested_score = cross_val_score(grid_search, X, y, cv=outer_cv, n_jobs=1, scoring='neg_mean_squared_error')

# Flip the negative sign to get positive MSE and then take the square root to get RMSE
nested_score_rmse = np.sqrt(-nested_score)

# Print the results
print(f"Nested CV RMSE: {nested_score_rmse.mean()} ± {nested_score_rmse.std()}")

/home/aws/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.017e+03, tolerance: 2.889e+01
  model = cd_fast.enet_coordinate_descent(
/home/aws/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.477e+03, tolerance: 2.820e+01
  model = cd_fast.enet_coordinate_descent(
/home/aws/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5

Nested CV RMSE: 27.775401231080156 ± 3.2739866888748668
